In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('RFM_ht_data.csv',sep=',')
df.head()

E:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [11]:
df.dtypes

InvoiceNo        object
CustomerCode     object
InvoiceDate      object
Amount          float64
dtype: object

In [12]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [16]:
df['CustomerCode'] = df['CustomerCode'].apply(str)
df['InvoiceNo'] = df['InvoiceNo'].apply(str)

In [17]:
df.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

In [18]:
last_date = df.InvoiceDate.max()

In [19]:
df.groupby('CustomerCode').agg({'InvoiceNo':'count'}).reset_index().sort_values('InvoiceNo',ascending=False)

,CustomerCode,InvoiceNo
89388,19057820,204
44594,13215452,113
10347,13032521,106
97077,19080880,99
119951,99003061,90
...,...,...
58910,13272861,1
58911,13272871,1
58913,13272875,1
58914,13272878,1


In [20]:

rfmTable = df.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)

In [21]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [30]:
RFM = rfmTable.reset_index()
RFM.head()

,CustomerCode,recency,frequency,monetary_value
0,02213019,19,1,1609.20
1,02213042,22,3,9685.48
2,02213071,29,1,415.00
3,02213088,23,1,305.00
4,02213092,25,1,1412.88


In [31]:
quantiles = RFM.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [32]:
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [36]:

RFM['R_Quartile'] = RFM['recency'].apply(RClass, args=('recency',quantiles))

RFM['F_Quartile'] = RFM['frequency'].apply(FMClass, args=('frequency',quantiles))

RFM['M_Quartile'] = RFM['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

RFM['RFMClass'] = RFM.R_Quartile.map(str) + RFM.F_Quartile.map(str) + RFM.M_Quartile.map(str)
RFM.head()

,CustomerCode,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
0,02213019,19,1,1609.20,4,4,3,443
1,02213042,22,3,9685.48,4,2,1,421
2,02213071,29,1,415.00,4,4,4,444
3,02213088,23,1,305.00,4,4,4,444
4,02213092,25,1,1412.88,4,4,3,443


In [44]:
RFM.value_counts(['RFMClass'])

RFMClass
444         10624
111          9705
443          6729
344          6593
211          5847
            ...  
424            63
214            60
114            60
314            33
414             2
Length: 64, dtype: int64

In [43]:
RFM.query("RFMClass == '311'").value_counts(['RFMClass'])

RFMClass
311         1609
dtype: int64